In [9]:
import pandas as pd

# Load historical food price data
df = pd.read_csv(r"D:\Python\foodprice\wfp_food_prices_ind.csv")

# Explore the data
print(df.head())

         date      admin1      admin2            market   latitude  longitude  \
0       #date  #adm1+name  #adm2+name  #loc+market+name   #geo+lat   #geo+lon   
1  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   
2  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   
3  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   
4  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   

             category      commodity        unit         priceflag  \
0          #item+type     #item+name  #item+unit  #item+price+flag   
1  cereals and tubers           Rice          KG            actual   
2  cereals and tubers          Wheat          KG            actual   
3  miscellaneous food          Sugar          KG            actual   
4        oil and fats  Oil (mustard)          KG            actual   

          pricetype   currency   price    usdprice  
0  #item+price+type  #currency  #value 

C:\Users\cp756\AppData\Local\Temp\ipykernel_2044\50839080.py:4: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"D:\Python\foodprice\wfp_food_prices_ind.csv")


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172018 entries, 0 to 172017
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   date       172018 non-null  object
 1   admin1     171229 non-null  object
 2   admin2     171229 non-null  object
 3   market     172018 non-null  object
 4   latitude   171229 non-null  object
 5   longitude  171229 non-null  object
 6   category   172018 non-null  object
 7   commodity  172018 non-null  object
 8   unit       172018 non-null  object
 9   priceflag  172018 non-null  object
 10  pricetype  172018 non-null  object
 11  currency   172018 non-null  object
 12  price      172018 non-null  object
 13  usdprice   172018 non-null  object
dtypes: object(14)
memory usage: 18.4+ MB


In [13]:
df.isnull().sum()

date           0
admin1       789
admin2       789
market         0
latitude     789
longitude    789
category       0
commodity      0
unit           0
priceflag      0
pricetype      0
currency       0
price          0
usdprice       0
dtype: int64

In [15]:
df.duplicated().sum()

0

In [16]:
df.describe()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
count,172018,171229,171229,172018,171229,171229,172018,172018,172018,172018,172018,172018,172018.0,172018.0000
unique,344,32,159,166,220,216,7,24,4,2,3,2,28483.0,43311.0000
top,2022-05-15,Uttar Pradesh,Mumbai city,Mumbai,18.975,72.825833,oil and fats,Rice,KG,actual,Retail,INR,20.0,0.2667
freq,3290,13173,4364,4364,2920,2920,41865,12430,160694,172017,169740,172017,1612.0,348.0000


In [18]:
df.fillna(10, inplace = True)

In [19]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional

In [22]:
import csv
import os

# File paths
train_path = r"D:\Python\foodprice\wfp_food_prices_ind.csv"
test_path = r"D:\Python\foodprice\wfp_food_prices_ind.csv"
desc_path = r"D:\Python\foodprice\wfp_food_prices_ind.csv"

# Check if files exist
if not all(map(os.path.exists, [train_path, test_path, desc_path])):
    raise FileNotFoundError("One or more files were not found. Please check the file paths.")

try:
    with open(train_path, 'r', encoding='utf-8') as train_file:
        train_reader = csv.reader(train_file)
        train_data = [row for row in train_reader]

    with open(test_path, 'r', encoding='utf-8') as test_file:
        test_reader = csv.reader(test_file)
        test_data = [row for row in test_reader]

    with open(desc_path, 'r', encoding='utf-8') as desc_file:
        descriptions = desc_file.readlines()

except FileNotFoundError as e:
    print(f"File not found: {e}")
except IOError as e:
    print(f"IO error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [23]:
all_descriptions = [item[3] for item in train_data] + [item[2] for item in test_data] + descriptions


In [24]:
labels = [item[2] for item in train_data] + [None for _ in test_data]

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [26]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(all_descriptions)

In [27]:
sequences = tokenizer.texts_to_sequences(all_descriptions)

In [28]:
# Pading sequences to a fixed length
max_sequence_length = 100 
sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [30]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [31]:
train_sequences = sequences[:len(train_data)]
test_sequences = sequences[len(train_data):len(train_data) + len(test_data)]

In [32]:
X_train, X_val, y_train, y_val = train_test_split(train_sequences, encoded_labels[:len(train_data)], test_size=0.2, random_state=42)

In [58]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

C:\Users\cp756\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [34]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [88]:
# Convert relevant columns to numeric
numeric_columns = ['latitude', 'longitude', 'price', 'usdprice', 'category', 'commodity', 'unit','priceflag'] 
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Drop non-numeric columns
X = df.drop(columns=['price', 'usdprice', 'date', 'admin1', 'admin2', 'market', 'category', 
                     'commodity', 'unit', 'priceflag', 'pricetype', 'currency'])

#features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# The target variable
y = df['price'].values

X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#path to save the best model
model_save_path = r'wfp_food_prices_ind.keras'

#ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath=model_save_path,
    monitor='val_loss',   
    save_best_only=True,       
    mode='min',            
    verbose=1                 
)

# Create an EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',        
    patience=3,              
    verbose=1,                 
    restore_best_weights=True  
)

# List of callbacks to be used during training
callbacks_list = [checkpoint, early_stopping]

# Model architecture
model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True, input_shape=(1, 6))),  
    Dropout(0.5),                                                      
    LSTM(32),                                                           
    Dropout(0.5),                                                       
    Dense(1, activation='linear')                                       
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])


# Print model summary
model.summary()

# Training the model
history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val),       
    epochs=50,                                       
    batch_size=32,                                 
    callbacks=callbacks_list                         
)

# After training, the best model is saved at 'Food_Price_Predictor.keras'
print(f"Best model saved at: {model_save_path}")


C:\Users\cp756\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_4 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
4297/4301 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 62700.9180 - mae: 76.6995
Epoch 1: val_loss did not improve from inf
4301/4301 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - loss: 62696.9375 - mae: 76.6932 - val_loss: nan - val_mae: nan
Epoch 2/50
4290/4301 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 56947.5742 - mae: 71.7449
Epoch 2: val_loss did not improve from inf
4301/4301 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 56946.8906 - mae: 71.7488 - val_loss: nan - val_mae: nan
Epoch 3/50
4299/4301 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 56766.0312 - mae: 75.8396
Epoch 3: val_loss did not improve from inf
4301/4301 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - loss: 56765.8945 - mae: 75.8397 - val_loss: nan - val_mae: nan
Epoch 3: early stopping
Restoring model weights from the end of the best epoch: 1.
Best model saved at: wfp_food_prices_ind.keras


In [90]:
model_save_path = r"D:\Python\wfp_food_prices_ind.keras"


In [92]:
model.save(model_save_path)


In [94]:
model.save(model_save_path, save_format='keras')


In [96]:
loaded_model = load_model(model_save_path)


In [98]:
# Save in HDF5 format
model.save(r'D:\Python\archive\Food_Price_Predictor.keras')

# Load the model
loaded_model = load_model(r'D:\Python\archive\Food_Price_Predictor.keras')


In [100]:
scaler_save_path = (r'D:\Python\archive\scaler.pkl')
joblib.dump(scaler, scaler_save_path)

categorical_columns = ['category', 'commodity', 'unit', 'pricetype']

label_encoder = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col]) 
    label_encoder[col] = le  
    
encoders_save_path = (r'D:\Python\archive\label_encoders.pkl')
joblib.dump(label_encoder,encoders_save_path)

['D:\\Python\\archive\\label_encoders.pkl']

In [102]:
# load the label encoders, scaler, and model
loaded_model = load_model(model_save_path)
loaded_scaler = joblib.load(scaler_save_path)
with open(encoders_save_path, 'rb') as f:
    loaded_label_encoders = joblib.load(f)

In [124]:
# Load pre-trained model, scaler, and encoders
loaded_model = load_model(model_save_path)
loaded_scaler = joblib.load(scaler_save_path)


with open(encoders_save_path, 'rb') as f:
    loaded_label_encoders = joblib.load(f)

# Define new data
new_data = {
    'latitude': 27.5706,
    'longitude': 80.0982,
    'category': 'cereals and tubers',
    'commodity': 'Rice',
    'unit': 'KG',
    'pricetype': 'Retail'
}

# Encode categorical variables properly
encoded_data = {}
for column in ['category', 'commodity', 'unit', 'pricetype']:
    if column in loaded_label_encoders:
        le = loaded_label_encoders[column] 
        if new_data[column] in le.classes_:
            encoded_data[column] = le.transform([new_data[column]])[0] 
        else:
            print(f"Warning: '{new_data[column]}' is a new category. Assigning unknown label (-1).")
            encoded_data[column] = -1  
    else:
        raise KeyError(f"Error: Encoder for '{column}' not found!")

# Add numerical features
encoded_data['latitude'] = new_data['latitude']
encoded_data['longitude'] = new_data['longitude']

# Convert to DataFrame (Ensure Correct Column Order)
new_data_df = pd.DataFrame([encoded_data])

# Make sure column names match the original training data
expected_columns = loaded_scaler.feature_names_in_  # Get original feature names
missing_cols = set(expected_columns) - set(new_data_df.columns)
extra_cols = set(new_data_df.columns) - set(expected_columns)


# Reorder columns to match the scaler's expected input
new_data_df = new_data_df[expected_columns]

# Scale the data
new_data_scaled = loaded_scaler.transform(new_data_df)

# Reshape input to be 3D [samples, timesteps, features]
new_data_scaled = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

# Predict
predicted_price = loaded_model.predict(new_data_scaled)

print(f"Predicted Price: {predicted_price[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 621ms/step
Predicted Price: 70.74281311035156


In [118]:
import pickle

# Load your trained model
model_path = r'D:\Python\archive\Food_Price_Predictor.keras'
loaded_model = load_model(model_path)

# Load your scaler
scaler_path = r'D:\Python\archive\scaler.save'
loaded_scaler = joblib.load(scaler_path)

# Create a dictionary with model details, including the model and scaler
model_details = {
    'model': loaded_model,  # Include the model in the dictionary
    'scaler': loaded_scaler,
    'description': 'LSTM model for food price prediction',
    'input_shape': loaded_model.input_shape,
}

# Save the details to a .pkl file
details_path =  r'D:\Python\archive\model_details.pkl'
with open(details_path, 'wb') as f:
    pickle.dump(model_details, f)

print("Model, details, and scaler saved to model_details.pkl")


Model, details, and scaler saved to model_details.pkl


In [120]:
import joblib
import gradio as gr
from tensorflow.keras.models import load_model

# Load the dictionary containing the model and other components
model_path = r'D:\Python\archive\model_details.pkl'
model_dict = joblib.load(model_path)

# Print the contents of the dictionary
print(model_dict)

# Check if the dictionary contains the required keys
if 'model' in model_dict and 'scaler' in model_dict:
    # Extract the model and scaler from the dictionary
    loaded_model = model_dict['model']
    loaded_scaler = model_dict['scaler']
else:
    print("Error: Dictionary does not contain the required keys.")
    exit()

# Define dropdown options
category_options = ["oils and fats", "cereals and tubers", "miscellaneous food", "vegetables and fruits"]
commodity_options = ["Wheat", "Rice", "Oil (mustard)", "Sugar", "Potatoes"]
unit_options = ["KG", "L", "100 KG"]
price_type_options = ["Wholesale", "Retail"]


# Prediction function
def predict(latitude, longitude, category, commodity, unit, pricetype):
    try:
        # Prepare input data for the model
        input_data = {
            'latitude': float(latitude),
            'longitude': float(longitude),
            'category': category_options.index(category),
            'commodity': commodity_options.index(commodity),
            'unit': unit_options.index(unit),
            'pricetype': price_type_options.index(pricetype)
        }
        input_df = pd.DataFrame([input_data])

        # Scale the input data
        input_scaled = loaded_scaler.transform(input_df)

        # Reshape input to be 3D [samples, timesteps, features]
        input_scaled = input_scaled.reshape((input_scaled.shape[0], 1, input_scaled.shape[1]))

        # Run the model prediction
        predicted_price = loaded_model.predict(input_scaled)
        return predicted_price[0][0]
    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio interface
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Number(label="Region (latitude)"),
        gr.Number(label="Region (longitude)"),
        gr.Dropdown(label="Category", choices=category_options),
        gr.Dropdown(label="Commodity", choices=commodity_options),
        gr.Dropdown(label="Unit", choices=unit_options),
        gr.Radio(label="Price Type", choices=price_type_options)
    ],
    outputs=gr.Textbox(label="Predicted Price"),
    title="Price Prediction of Agri-Horticultural Commodities",
    description="Enter the following to get the predicted price."
)

# Launch the interface
interface.launch()

{'model': <Sequential name=sequential_4, built=True>, 'scaler': MinMaxScaler(), 'description': 'LSTM model for food price prediction', 'input_shape': (None, 1, 2)}
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
